In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Sequential
from keras.layers import LSTM,Dense,Dropout,Embedding,CuDNNLSTM,Bidirectional
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Using TensorFlow backend.


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

df = pd.read_csv('unprocessed_lyrics.csv')
df = df.groupby('Genre', group_keys=False).apply(lambda s: s.sample(5000, random_state=42)) #


In [3]:
df

,SName,Lyric,Genre,Artist,lyric_length
2157,Family Affair,Refrain:. Let's get it crunkupon. We gon' have...,Hip Hop,Mary J. Blige,567
3187,Ransom (ft. Lil' Wayne),"Ransom,. . Yeah,. its Drizzy Baby. you already...",Hip Hop,Drake,967
9164,Close To Me,"T.O.S.. (50 Cent). Unstoppable, incredible, im...",Hip Hop,G-Unit,557
2145,Zone,"uhh, yea. uh uh uh. alright, well alright. . i...",Hip Hop,Drake,400
6442,"Why You Up In Here (feat. Ludacris, Git Fresh ...",Flo-Rida. Gucci!. Bird!. I done bought all thi...,Hip Hop,Flo Rida,524
...,...,...,...,...,...
31738,What do You Need?,What do you need from me tonight?. I feel you ...,Rock,Goo Goo Dolls,197
31425,Rebel Heart,"(R. Stewart, J. Golub, C. Kentis, C. Rojas). I...",Rock,Rod Stewart,412
35407,Before The Dawn,Meet me after dark again and I'll hold you. I ...,Rock,Evanescence,114
35281,Spanish is the Loving Tongue,"Broke my heart, lost my soul. Adios,mi cora so...",Rock,Bob Dylan,126


In [4]:
x = df['Lyric']
y = df['Genre']

In [5]:
MAXLENGTH = 500

In [25]:
label_encoder = preprocessing.LabelEncoder()
labels = label_encoder.fit_transform(df.Genre.values)
np.unique(labels)

In [26]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest = train_test_split(df.Lyric.values, 
                                                labels, 
                                                random_state=42, 
                                                test_size=0.1, 
                                                shuffle=True)

In [27]:
token = Tokenizer()


In [9]:
token.fit_on_texts(xtrain)
seq = token.texts_to_sequences(xtrain)


In [10]:
pad_seq = pad_sequences(seq,maxlen=MAXLENGTH)


In [11]:
seq_test = token.texts_to_sequences(xtest)
pad_seq_test = pad_sequences(seq_test,maxlen=MAXLENGTH)


In [12]:
vocab_size = len(token.word_index)+1


In [13]:
embedding_vector = {}
f = open('glove.twitter.27B.200d.txt',encoding='utf8')
for line in tqdm(f):
    value = line.split(' ')
    word = value[0]
    coef = np.array(value[1:],dtype = 'float32')
    embedding_vector[word] = coef

1193517it [00:59, 19985.88it/s]


In [14]:
embedding_matrix = np.zeros((vocab_size,200))
for word,i in tqdm(token.word_index.items()):
    embedding_value = embedding_vector.get(word)
    if embedding_value is not None:
        embedding_matrix[i] = embedding_value

100%|██████████| 59210/59210 [00:00<00:00, 427125.29it/s]


In [15]:
model=Sequential()

In [16]:
model.add(Embedding(vocab_size,200,weights = [embedding_matrix],input_length=MAXLENGTH,trainable = False))


In [17]:
model.add(Bidirectional(LSTM(128, dropout=0.3, recurrent_dropout=0.3)))



Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [18]:
model.add(Dense(32,activation = 'relu'))


In [19]:
model.add(Dense(3,activation = 'sigmoid'))


In [20]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics = ['accuracy'])




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [21]:
pad_seq.shape

(13500, 500)

In [22]:
from keras.utils import np_utils

ytrain_encode = np_utils.to_categorical(ytrain)
ytest_encode = np_utils.to_categorical(ytest)

In [23]:
history = model.fit(pad_seq,ytrain_encode,epochs = 2,batch_size=16)




Epoch 1/2
13500/13500 [==============================] - 1067s 79ms/step - loss: 0.5089 - acc: 0.7352
Epoch 2/2
13500/13500 [==============================] - 1067s 79ms/step - loss: 0.4689 - acc: 0.7576


Large diff between acc on training vs testing (might be overfit)?

In [24]:
from sklearn.metrics import classification_report

y_pred = model.predict(pad_seq_test, batch_size=16, verbose=1)
y_pred_bool = np.argmax(y_pred, axis=1)
print(classification_report(ytest, y_pred_bool))

1500/1500 [==============================] - 47s 31ms/step
              precision    recall  f1-score   support

           0       0.82      0.84      0.83       516
           1       0.54      0.56      0.55       503
           2       0.62      0.58      0.60       481

    accuracy                           0.66      1500
   macro avg       0.66      0.66      0.66      1500
weighted avg       0.66      0.66      0.66      1500

